Gentry's Somewhat Homomorphic Cipher Demo
=======================================

Note for using Jupyter: Use $\texttt{SHIFT + ENTER}$ to execute the code snippets!

Recall the __Ideal Coset Problem__:

Given a security parameter $\lambda = 1^n$, a ring $R$ with an ideal $I$, a PPT algorithm $\texttt{Samp}_R$ that samples $R$ according to some distribution, a PPT algorithm $\texttt{GenIdeal}$ and a PPT adversary $\mathcal{A}$, the ICP experiment is as follows:

1. Run $\texttt{GenIdeal}(\lambda, R, I)$ and obtain a tuple $(J^{pk}, J^{sk})$, two generators of the same ideal $J$ such that $I + J = R$.
2. Generate $r_0 \leftarrow \texttt{Samp}_R$ and $r_1 \leftarrow R$ and a bit $b \leftarrow \{0, 1\}$. Then, calculate $r = r_b \mod J^{pk}$.
3. $\mathcal{A}$ is given $R, I, J^{pk}, \texttt{Samp}_R$ and $r$ and outputs some bit $b' \in \{0, 1\}$.
4. Set $\texttt{Expr}^{ICP}_{\mathcal{A}, R, I, \texttt{Samp}_R, \texttt{GenIdeal}}(n) = 1$ if $b = b'$ and 0 otherwise.

Informally, the ICP asks an adversary to decide whether the coset representative it receives was picked uniformly randomly, or whether it was picked according to some different distribution (defined by $\texttt{Samp}_R$). This would be impossible if the custom sampling function was also uniform, but the hardness assumption is that there are non-uniform sampling algorithms for which it is still hard to succeed.

Formally, the __ICP hardness assumption__ says, that the ICP is hard relative to $(R, I ,\texttt{Samp}_R, \texttt{GenIdeal})$ if every PPT adversary $\mathcal{A}$ has negligible probability to succeed, i.e.

$$
\texttt{Adv}^{ICP}_{\mathcal{A}, R, I, \texttt{Samp}_R, \texttt{GenIdeal}}(n) = \left| 2 \cdot \texttt{Expr}^{ICP}_{\mathcal{A}, R, I, \texttt{Samp}_R, \texttt{GenIdeal}}(n) - 1 \right|\leq \text{negl}(n).
$$

The above might seem really far removed, but it has two advantages:
1. It yields a straightforward trapdoor, namely the ideal generators $J^{pk}, J^{sk}$ which will serve as the public and private key.
2. We can build a scheme based on the ICP and then prove its security based on it. Then, we can "substitute in" something more concrete and be guaranteed a secure scheme! (See Section 9.2.3 in the report.)

Now, we pick $R = \mathbb{Z}[x]/f$, where $f(x) = x^N + 1$ for $\lambda = 1^n$ and $N = 2^n$. It can be shown that $f$ is irreducible and hence $R$ is a field. Pick $I = (2) \leq R$. Then, define $\texttt{GenIdeal}$ so that it picks some $v \in R$ with its coefficients being between some parameter $t$, such that $v \not\in I$. Also define $\texttt{Samp}_R$ to pick $u \in R$ such that its coefficients are picked from $\{0, 1\}$, with 0 having some probability $p$ and 1 with probability $1 - p$. This will mean that the norm of $u$ will be extremely small.

In [7]:
load('somewhat_homom_improved.sage')

In [8]:
pk, sk = key_gen(N=7, t = 128)

HNF in correct form after 2 tries.


In [10]:
m1 = 1
m2 = 1
m3 = 0

c1 = encrypt(pk, m1)
c2 = encrypt(pk, m2)
c3 = encrypt(pk, m3)

Perturbation is 128 dimensional and has 11 non-zero entries.
Namely, u(x) = -xbar^119 - xbar^110 - xbar^104 - xbar^99 + xbar^95 + xbar^94 + xbar^85 - xbar^33 + xbar^29 + xbar^16 + xbar^14
Perturbation is 128 dimensional and has 7 non-zero entries.
Namely, u(x) = xbar^124 - xbar^118 - xbar^101 + xbar^92 + xbar^77 - xbar^76 - xbar^40
Perturbation is 128 dimensional and has 8 non-zero entries.
Namely, u(x) = xbar^115 - xbar^110 - xbar^108 - xbar^86 - xbar^75 + xbar^37 + xbar^24 + xbar^14


In [ ]:
decrypt(sk, c1)